## 俳句で使われている単語について共起性を可視化する

### 参考資料

- [pyvis公式サイト](https://pyvis.readthedocs.io/en/latest/tutorial.html)
- [[自然言語処理/NLP] pyvisライブラリを使って共起ネットワークを簡単に描画してみる(SageMaker使用)](https://dev.classmethod.jp/articles/mrmo-20190930/)

In [3]:
import pandas as pd
import itertools
import collections

In [4]:
df = pd.read_pickle('../../data/pickles/haiku_df.pickle')
df.head()

,作者,フルネーム,俳句,季語,季節,分ち書き,名詞,動詞
0,bashou,松尾芭蕉,姥桜さくや老後の思ひ出,桜,春,姥桜 さく や 老後 の 思ひ出,姥桜 老後 思ひ出,咲く
1,bashou,松尾芭蕉,年は人にとらせていつも若夷,NaN,NaN,年 は 人 に とら せ て いつ も 若 夷,年 人 若 夷,取る
2,bashou,松尾芭蕉,花の顔に晴うてしてや朧月,花,春,花 の 顔 に 晴 うて し て や 朧月,花 顔 朧月,晴れる 打てる 為る
3,bashou,松尾芭蕉,盛なる梅にす手引風も哉,梅,春,盛 なる 梅 に す 手引 風 も 哉,梅 手引 風,為る
4,bashou,松尾芭蕉,あち東風や面々さばき柳髪,東風,春,あち 東風 や 面々 さばき 柳 髪,東風 面々 柳 髪,捌く


In [5]:
# 俳人の選定
haijin = ['松尾芭蕉', '与謝蕪村', '小林一茶', '正岡子規', '夏目漱石']
selected_df = df[df['フルネーム'].isin(haijin)].copy()
selected_df

,作者,フルネーム,俳句,季語,季節,分ち書き,名詞,動詞
0,bashou,松尾芭蕉,姥桜さくや老後の思ひ出,桜,春,姥桜 さく や 老後 の 思ひ出,姥桜 老後 思ひ出,咲く
1,bashou,松尾芭蕉,年は人にとらせていつも若夷,NaN,NaN,年 は 人 に とら せ て いつ も 若 夷,年 人 若 夷,取る
2,bashou,松尾芭蕉,花の顔に晴うてしてや朧月,花,春,花 の 顔 に 晴 うて し て や 朧月,花 顔 朧月,晴れる 打てる 為る
3,bashou,松尾芭蕉,盛なる梅にす手引風も哉,梅,春,盛 なる 梅 に す 手引 風 も 哉,梅 手引 風,為る
4,bashou,松尾芭蕉,あち東風や面々さばき柳髪,東風,春,あち 東風 や 面々 さばき 柳 髪,東風 面々 柳 髪,捌く
...,...,...,...,...,...,...,...,...
7545,souseki,夏目漱石,杉木立寺を蔵して時雨けり,時雨,冬,杉 木立 寺 を 蔵し て 時雨 けり,杉 木立 寺,隠す 時雨れる
7546,souseki,夏目漱石,豆腐焼く串にはらはら時雨哉,時雨,冬,豆腐 焼く 串 に はらはら 時雨 哉,豆腐 串 時雨,焼く
7547,souseki,夏目漱石,内陣に佛の光る寒哉,NaN,NaN,内陣 に 佛 の 光る 寒 哉,内陣 佛,光る
7548,souseki,夏目漱石,水仙や早稲田の師走三十日,水仙,冬,水仙 や 早稲田 の 師走 三十 日,水仙 早稲田 師走 三十,


In [10]:
combination_dict = {}
for key, group in selected_df.groupby('作者'):
    words_list = group['名詞'].tolist()
    words_list = [ words.split() for words in words_list ]
    word_combs = [list(itertools.combinations(words, 2)) for words in words_list ]
    word_combs = [[tuple(sorted(words)) for words in comb] for comb in word_combs]
    target_combs = []
    for combs in word_combs:
        target_combs.extend(combs)
#     combination_dict[key] = target_combs

[[('姥桜', '老後'), ('姥桜', '思ひ出'), ('思ひ出', '老後')], [('人', '年'), ('年', '若'), ('夷', '年'), ('人', '若'), ('人', '夷'), ('夷', '若')], [('花', '顔'), ('朧月', '花'), ('朧月', '顔')], [('手引', '梅'), ('梅', '風'), ('手引', '風')], [('東風', '面々'), ('東風', '柳'), ('東風', '髪'), ('柳', '面々'), ('面々', '髪'), ('柳', '髪')], [('雪', '餅'), ('しら糸', '餅'), ('柳', '餅'), ('しら糸', '雪'), ('柳', '雪'), ('しら糸', '柳')], [('嘆', '花'), ('うた', '花'), ('ぶくろ', '花'), ('うた', '嘆'), ('ぶくろ', '嘆'), ('うた', 'ぶくろ')], [('なつ', '口'), ('たば', 'なつ'), ('なつ', '花'), ('なつ', '風'), ('たば', '口'), ('口', '花'), ('口', '風'), ('たば', '花'), ('たば', '風'), ('花', '風')], [('人', '初瀬'), ('人', '山桜'), ('初瀬', '山桜')], [('かへるさ', '糸桜'), ('糸桜', '足'), ('かへるさ', '足')], [('尾', '風'), ('犬', '風'), ('櫻', '風'), ('尾', '犬'), ('尾', '櫻'), ('櫻', '犬')], [('わらは', '春'), ('かざり', '春'), ('春', '縄'), ('かざり', 'わらは'), ('わらは', '縄'), ('かざり', '縄')], [('羽織', '花'), ('ごろ', '羽織'), ('ごろ', '花')], [('世', '花'), ('花', '間口'), ('花', '風'), ('世', '間口'), ('世', '風'), ('間口', '風')], [('事', '子'), ('事', '児'), ('事', '櫻'), ('児', '子'), ('子', '櫻')

In [24]:
for key, value in combination_dict.items():
    ct = collections.Counter(combination_dict[key])
    print('{}\n'.format(key), ct.most_common()[:10], '\n')

bashou
 [(('秋', '風'), 15), (('梅', '花'), 7), (('暮', '秋'), 6), (('三', '月'), 6), (('世', '花'), 5), (('夏', '月'), 5), (('けふ', '月'), 5), (('今宵', '月'), 5), (('あさま', '石'), 5), (('花', '雪'), 4)] 

buson
 [(('暮', '秋'), 11), (('春', '雨'), 8), (('夜', '月'), 7), (('夏', '月'), 6), (('秋', '風'), 6), (('冬', '木立'), 6), (('春', '水'), 5), (('人', '春'), 5), (('夜', '春'), 5), (('夏', '木立'), 5)] 

issa
 [(('秋', '風'), 13), (('暮', '秋'), 13), (('春', '雨'), 12), (('峰', '雲'), 11), (('夜', '山'), 10), (('乙', '鳥'), 9), (('花', '陰'), 8), (('艸', '花'), 7), (('夏', '木立'), 6), (('春', '月'), 6)] 

siki
 [(('秋', '風'), 24), (('三', '月'), 13), (('山', '秋'), 12), (('けふ', '月'), 11), (('岸', '根'), 10), (('山', '雲'), 10), (('夏', '月'), 10), (('暮', '秋'), 10), (('人', '秋'), 10), (('人', '春'), 9)] 

souseki
 [(('秋', '空'), 15), (('山', '秋'), 9), (('秋', '雨'), 9), (('水', '音'), 8), (('秋', '風'), 8), (('峰', '雲'), 7), (('影', '月'), 7), (('秋', '雲'), 7), (('影', '法師'), 6), (('石蕗', '花'), 6)] 



## 共起ネットワークの描画

In [21]:
from pyvis.network import Network

In [26]:
# 上記のctをcsvに保存
for key, value in combination_dict.items():
    ct = collections.Counter(combination_dict[key])
#     save to csv
#     pd.DataFrame([{'word1': i[0][0], 'word2': i[0][1], 'count': i[1]} for i in ct.most_common()]).to_csv('./data/co_occurrence_{}.csv'.format(key), index=False)

In [29]:
def vis_co_occurrence(haijin):
    got_net = Network(height='750px', width="100%", bgcolor="#222222", font_color="white", notebook=True)

    # set the physics layout of the network
    got_net.barnes_hut()
    got_data = pd.read_csv("./data/co_occurrence_{}.csv".format(haijin))

    sources = got_data['word1']
    targets = got_data['word2']
    weights = got_data['count']

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        wei = e[2]

        got_net.add_node(src, src, title=src)
        got_net.add_node(dst, dst, title=dst)
        got_net.add_edge(src, dst, value=wei)

    neighbour_map = got_net.get_adj_list()

    for node in got_net.nodes:
        node["title"] += " Neighbour:<br>" + "<br>".join(neighbour_map[node["id"]])
        node["value"] = len(neighbour_map[node["id"]])

    got_net.show("./data/vis_co_occurrence_{}.html".format(haijin))

In [1]:
# ネットワークの可視化
# for name in combination_dict.keys():
#     vis_co_occurrence(name)